In [1]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# read in postcodes
postcodes = pd.read_csv("./POAs.csv", usecols=["POA_NAME_2016"], dtype={"POA_NAME_2016": "str"})
postcodes = list(postcodes["POA_NAME_2016"].unique())

# use a headless browser (saves time)
chrome_options = Options()
chrome_options.add_argument("--headless")

# set up the browser
driver = webdriver.Chrome(options=chrome_options)
business_directory = pd.DataFrame(columns=["name", "ABN", "ACN"])

# navigate to search page
def begin_search():
    driver.get("https://ibd.supplynation.org.au/public/s/search-results")
    data_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_location((By.CSS_SELECTOR, ".slds-media:nth-child(3) .slds-media__body"))
    )
    name = data_field.find_element(By.ELEMENT_NAME, "span").get_attribute("innerText")
    abn = data_field.find_element(By.CSS_SELECTOR, "p:nth-child(2) span:last-child").get_attribute("innerText")
    abn = data_field.find_element(By.CSS_SELECTOR, "p:nth-child(2) span:last-child").get_attribute("innerText")
    
    
    
    document.querySelector(".slds-media:nth-child(3) .slds-media__body p:nth-child(2) span:last-child")
    
    
# enter postcode and select first option
def enter_postcode(postcode):
    print("Scraping for postcode", postcode, "...")
    global previousPage
    search_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "snsm-location-input"))
    )
    search_field.clear()
    search_field.send_keys(postcode[0])
    time.sleep(.25)
    search_field.send_keys(postcode[1])
    time.sleep(.25)
    search_field.send_keys(postcode[2])
    time.sleep(.25)
    search_field.send_keys(postcode[3])
    
    time.sleep(2)
    # gather location suggestions
    location_suggestions = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "slds-listbox__option-text"))
    )    
    location_suggestions[0].click()
    submit_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "searchbtn"))
    )
    submit_button.click()
    
    previousPage = 0
    business_pages(previousPage)

def business_pages(page):
    currentPage = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".uiOutputText:nth-child(2)"))
    )
    time.sleep(2)
    currentPage = int(currentPage.get_attribute("innerText"))    
    
    if currentPage > page:
        print("page:", page)
        previousPage = currentPage        
        gather_businesses(currentPage)
    else:
        print("Scraping complete for postcode", postcode, "— business directory now has", len(business_directory), "entries.")        

def gather_businesses(page):
    businesses = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".slds-text-heading_medium a"))
    )
    for business in businesses:        
        business_id = business.get_attribute("data-supplierid")
        if business_id not in business_directory:
            business_directory.append(business_id)
    next_button(page)
    
def next_button(page):
    next = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".slds-button-group button:last-child"))                
    )
    next.click()
    business_pages(page)

for postcode in postcodes:
    begin_search()
    enter_postcode(postcode)

Scraping for postcode 0800 ...
page: 0
page: 1
page: 10
page: 19
page: 28
page: 37
page: 46
page: 55
page: 64
page: 73
page: 82
page: 91
page: 100
page: 109
page: 118
page: 127
page: 136
page: 145
page: 154
page: 163
page: 172
page: 181
page: 190
Scraping complete for postcode 0800 — business directory now has 200 entries.
Scraping for postcode 0810 ...
page: 0
page: 1
page: 10
page: 19
page: 28
page: 37
page: 46
page: 55
page: 64
page: 73
page: 82
page: 91
page: 100
page: 109
page: 118
page: 127
page: 136
page: 145
page: 154
page: 163
page: 172
page: 181
page: 190
Scraping complete for postcode 0810 — business directory now has 200 entries.
Scraping for postcode 0812 ...
page: 0
page: 1
page: 10
page: 19
page: 28
page: 37
page: 46
page: 55
page: 64
page: 73
page: 82
page: 91
page: 100
page: 109
page: 118
page: 127
page: 136
page: 145
page: 154
page: 163
page: 172
page: 181
page: 190
Scraping complete for postcode 0812 — business directory now has 200 entries.
Scraping for postcode 081

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: headless chrome=86.0.4240.75)


In [17]:
print(len(business_directory))
postcodes.index("6327")

2559


2354

In [20]:
pd.Series(business_directory).to_csv("business.csv")